In [8]:
!pip install opencv-python-headless numpy --user

In [3]:
import os
import shutil
import random

# Define the paths to your dataset folders
dataset_dir = "dataset"  # Update this to the correct path
training_dir = "training"
testing_dir = "testing"
validating_dir = "validating"

# Define the ratio for splitting (80% training, 10% testing, 10% validating)
train_ratio = 0.8
test_ratio = 0.1
valid_ratio = 0.1

# Create the destination folders if they don't exist
os.makedirs(training_dir, exist_ok=True)
os.makedirs(testing_dir, exist_ok=True)
os.makedirs(validating_dir, exist_ok=True)

# Iterate through the "car" and "notcar" folders
categories = ["car", "notcar"]
for category in categories:
    category_dir = os.path.join(dataset_dir, category)  # Construct the full path to the category folder
    
    # Create subdirectories in the destination folders for "car" and "notcar"
    os.makedirs(os.path.join(training_dir, category), exist_ok=True)
    os.makedirs(os.path.join(testing_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validating_dir, category), exist_ok=True)
    
    # Get a list of all image files in the category folder
    image_files = os.listdir(category_dir)
    
    # Shuffle the image files randomly
    random.shuffle(image_files)
    
    # Calculate the number of images for each split
    num_images = len(image_files)
    num_train = int(num_images * train_ratio)
    num_test = int(num_images * test_ratio)
    
    # Split the images into training, testing, and validating sets
    train_images = image_files[:num_train]
    test_images = image_files[num_train:num_train + num_test]
    valid_images = image_files[num_train + num_test:]
    
    # Copy the images to their respective folders
    for image in train_images:
        src = os.path.join(category_dir, image)
        dst = os.path.join(training_dir, category, image)
        shutil.copy(src, dst)
    
    for image in test_images:
        src = os.path.join(category_dir, image)
        dst = os.path.join(testing_dir, category, image)
        shutil.copy(src, dst)
    
    for image in valid_images:
        src = os.path.join(category_dir, image)
        dst = os.path.join(validating_dir, category, image)
        shutil.copy(src, dst)

print("Data split and copied successfully.")


Data split and copied successfully.


In [12]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(0)

# Define data directories
car_folder = "training/car"
not_car_folder = "training/notcar"

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load the VGG16 model with pre-trained weights (excluding the top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Prepare data generators
train_generator = train_datagen.flow_from_directory(
    directory="training",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['notcar', 'car'])

# Train the model
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=1,
        verbose=1)
    
    # Print accuracy and loss after each epoch
    print(f"Accuracy: {history.history['accuracy'][0]:.2f}")
    print(f"Loss: {history.history['loss'][0]:.2f}")

# Save the trained model
model.save("car_detection_model.keras")


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [2]:
# Function to predict and save images to output directories
def predict_and_save_images(image_folder, output_folder, label):
    for image_name in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_name)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        
        # Normalize pixel values
        img_array /= 255.0
        
        # Predict class (car or not car)
        prediction = model.predict(img_array)[0][0]
        
        # Classify as car if prediction is above 0.5, else classify as not car
        if prediction > 0.5 and image_folder==valid_car_folder :
            output_path = os.path.join(output_folder, image_name)
            img.save(output_path)
        elif prediction <= 0.5 and image_folder==valid_not_car_folder :
            output_path = os.path.join(output_folder, image_name)
            img.save(output_path)

**Validating The Model**

In [4]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

# Define data directories
valid_car_folder = "validating/car"
valid_not_car_folder = "validating/notcar"
output_car_folder = "validating/outputcar"
output_not_car_folder = "validating/outputnoncar"

# Load the trained model
model = load_model("car_detection_model.keras")

# Create output directories if they don't exist
os.makedirs(output_car_folder, exist_ok=True)
os.makedirs(output_not_car_folder, exist_ok=True)

# Predict and save car images
predict_and_save_images(valid_car_folder, output_car_folder, label="car")

# Predict and save non-car images
predict_and_save_images(valid_not_car_folder, output_not_car_folder, label="notcar")

# Evaluate the model on validation data
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
validation_generator = validation_datagen.flow_from_directory(
    directory="validating",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False,
    classes=['notcar', 'car']
)

validation_loss, validation_accuracy = model.evaluate(validation_generator)

# Print validation results
print(f"Validation Loss: {validation_loss:.2f}")
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")


1/1 [==============================] - 0s 365ms/step
Found 208 images belonging to 2 classes.
7/7 [==============================] - 42s 5s/step - loss: 0.0830 - accuracy: 0.9712
Validation Loss: 0.08
Validation Accuracy: 97.12%


**Testing the model**

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Define data directories
test_car_folder = "testing/car"
test_not_car_folder = "testing/notcar"
output_test_car_folder = "testing/outputcar"
output_test_not_car_folder = "testing/outputnoncar"

# Load the trained model
model = load_model("car_detection_model.keras")

# Create output directories if they don't exist
os.makedirs(output_test_car_folder, exist_ok=True)
os.makedirs(output_test_not_car_folder, exist_ok=True)

    
# Predict and save car images
predict_and_save_images(test_car_folder, output_test_car_folder, label="car")

# Predict and save non-car images
predict_and_save_images(test_not_car_folder, output_test_not_car_folder, label="notcar")

# Evaluate the model on testing data
testing_datagen = ImageDataGenerator(rescale=1.0 / 255)
testing_generator = testing_datagen.flow_from_directory(
    directory="testing",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False,
    classes=['notcar', 'car']
)

testing_loss, testing_accuracy = model.evaluate(testing_generator)

# Print testing results
print(f"Testing Loss: {testing_loss:.2f}")
print(f"Testing Accuracy: {testing_accuracy * 100:.2f}%")


**Experiment**

In [4]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Define data directories
test_car_folder = "testing/car"
test_not_car_folder = "testing/notcar"
output_test_car_folder = "testing/outputcar"
output_test_not_car_folder = "testing/outputnoncar"

# Load the trained model
model = load_model("car_detection_model.keras")

# Create output directories if they don't exist
os.makedirs(output_test_car_folder, exist_ok=True)
os.makedirs(output_test_not_car_folder, exist_ok=True)

# Function to predict and save images with filled green rectangle
def predict_and_save_images(input_folder, output_folder, label):
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img_array = np.expand_dims(img, axis=0) / 255.0

        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)

        if label == "car" and predicted_class == 1:
            # Car detected, draw a filled green rectangle
            img = cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), (0, 255, 0), -1)

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

# Predict and save car images
predict_and_save_images(test_car_folder, output_test_car_folder, label="car")

# Predict and save non-car images
predict_and_save_images(test_not_car_folder, output_test_not_car_folder, label="notcar")

# Evaluate the model on testing data
testing_datagen = ImageDataGenerator(rescale=1.0 / 255)
testing_generator = testing_datagen.flow_from_directory(
    directory="testing",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False,
    classes=['notcar', 'car']
)

testing_loss, testing_accuracy = model.evaluate(testing_generator)

# Print testing results
print(f"Testing Loss: {testing_loss:.2f}")
print(f"Testing Accuracy: {testing_accuracy * 100:.2f}%")


1/1 [==============================] - 0s 199ms/step
Found 204 images belonging to 2 classes.
7/7 [==============================] - 39s 5s/step - loss: 0.1076 - accuracy: 0.9755
Testing Loss: 0.11
Testing Accuracy: 97.55%


**Testing Via Video**

In [8]:
import cv2
import shutil
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("car_detection_model.keras")

# Open the video file
video_capture = cv2.VideoCapture(r"D:\College\Sem 5\NFT\Assignment2\input3.mp4")

# Create output directories if they don't exist
output_car_folder = "testvideo/outputcar"
output_not_car_folder = "testvideo/outputnotcar"
os.makedirs(output_car_folder, exist_ok=True)
os.makedirs(output_not_car_folder, exist_ok=True)

# Function to predict and save frames to output directories
def predict_and_save_frames(frame, output_folder, frame_number):
    frame = cv2.resize(frame, (224, 224))
    img_array = img_to_array(frame)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Normalize pixel values
    img_array /= 255.0
    
    # Predict class (car or not car)
    prediction = model.predict(img_array)[0][0]
    
    # Determine the label based on the prediction
    if prediction > 0.5:
        label = "car"
        output_path = os.path.join(output_car_folder, f"frame_{frame_number:04d}_{label}.png")
        cv2.imwrite(output_path, frame)
    else:
        label = "notcar"
        output_path = os.path.join(output_not_car_folder, f"frame_{frame_number:04d}_{label}.png")
        cv2.imwrite(output_path, frame)
    
    # Save the frame with a unique filename
    #output_path = os.path.join(output_folder, f"frame_{frame_number:04d}_{label}.png")
    #cv2.imwrite(output_path, frame)

frame_count = 0
while True:
    # Read a frame from the video
    ret, frame = video_capture.read()
    if not ret:
        print("End of video.")
        break  # Break the loop if we reach the end of the video
    
    frame_count += 1
    
    # Classify the frame and save it to the appropriate output folder
    predict_and_save_frames(frame, output_car_folder, frame_count)
    predict_and_save_frames(frame, output_not_car_folder, frame_count)

    # Press 'q' to exit the loop and close the window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
video_capture.release()
cv2.destroyAllWindows()

print(f"Processed {frame_count} frames.")

1/1 [==============================] - 1s 719ms/step
End of video.
Processed 241 frames.
